In [ ]:
import os
from rqalpha_plus import run_func
from rqalpha_plus.api import *
import numpy as np


config = {
    'extra': {
        'log_level': 'error',
    },
    'base': {
        'matching_type': 'current_bar',
        'start_date': '2015-01-01',
        'end_date': '2018-03-27',
        'benchmark': '000001.XSHE',
        'frequency': '1d',
        'accounts': {
                 'future': 1000000
             }
    },
    'mod': {
        'sys_progress': {
            'enabled': True,
            'show': True,
        },
        'sys_analyser': {
            'enabled': True,
        },
    },
}

tasks = []
for k1 in np.arange(0.3, 1, 0.1):
    for k2 in np.arange(0.3, 1, 0.1):
      for periods in np.arange(3,10,1):
        tasks.append((config, k1, k2,periods))


def run_bt(config, k1, k2,periods):
    import talib

    # 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。

    # 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
    def init(context):
        # context内引入全局变量s1
        context.futures_marks = "J"
        context.fired = False
        context.observed_periods = periods
        context.K1 = k1
        context.K2 = k2
        context.s1 = get_dominant_future(context.futures_marks,end_date=context.now)[-1]
        context.long_signal = False
        context.short_signal = False
        print(context.s1)
        # 初始化时订阅合约行情。订阅之后的合约行情会在handle_bar中进行更新。
        subscribe(context.s1)
        # 实时打印日志

    # before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
    def before_trading(context):
        context.long_signal = False
        context.short_signal = False
        context.daily_open_price = history_bars(context.s1,2,'1d','open')[-1]
        # plot("open_price",context.daily_open_price)


    # 你选择的期货数据更新将会触发此段逻辑，例如日线或分钟线更新
    def handle_bar(context, bar_dict):
        # 每日开盘的开盘价
        change_contract(context)
        handling_commands(context,bar_dict)


    def calc_Points(context,bar_dict):
        _price = history_bars(context.s1,context.observed_periods,'1d',['close','high','low','open'])
        HH = _price['high'][:-1].max()
        LC = _price['close'][:-1].min()

        HC = _price['close'][:-1].max()
        LL = _price['low'][:-1].min()

        ranges = max(HH-LC,HC-LL)

        # 获得两个上下界
        _open = context.daily_open_price 
        up_point = _open + context.K1 * ranges
        low_point = _open - context.K2 * ranges
        plot("up",up_point)
        plot("down",low_point)

        return up_point,low_point

    def handling_commands(context,bar_dict):
        up_point,low_point = calc_Points(context,bar_dict)
        bqt = context.portfolio.positions[context.s1].buy_quantity
        sqt = context.portfolio.positions[context.s1].sell_quantity

        # 开多仓
        if bar_dict[context.s1].last > up_point :
            print("开多仓")
            if sqt >0:
                buy_close(context.s1,sqt)
            if not context.long_signal:
                context.long_signal = True
                buy_open(context.s1,1)

        # 开空仓
        if bar_dict[context.s1].last < low_point :
            print("开空仓")
            if bqt > 0:
                sell_close(context.s1,bqt)
            if not context.short_signal:
                sell_open(context.s1,1)
                context.short_signal = True

    def change_contract(context):
        dominant_future = get_dominant_future(context.futures_marks,end_date=context.now)[-1]
        if not context.s1 == dominant_future:
            print("强制平仓")
            clear_up(context)
            context.s1 = dominant_future    
            subscribe(context.s1)


    def clear_up(context):
        _positions = context.portfolio.positions[context.s1]
        bqt = _positions.buy_quantity
        sqt = _positions.sell_quantity

        if bqt>0:
            sell_close(context.s1,bqt)
        elif sqt>0:
            buy_close(context.s1,sqt)
    

    # after_trading函数会在每天交易结束后被调用，当天只会被调用一次
    def after_trading(context):
        pass
            
            
    name = '{}-{}-{}'.format(k1, k2,periods)
    result = run_func(config=config, init=init, handle_bar=handle_bar,before_trading=before_trading)
    return result, name


results = []
for task in tasks:
    result, name = run_bt(*task)
    results.append((result['sys_analyser'], name))

    
# 从回测结果中提取分析信息
data = []
for result, name in results:
    summary = result['summary']
    data.append({
        'name': name,
        'annualized_returns': summary['annualized_returns'],
        'sharpe': summary['sharpe'],
        'max_drawdown': summary['max_drawdown'],
    })    
    
    
# 分析回测
results_df = pd.DataFrame(data)
 
print('')
print('-' * 50)
print('Sort by sharpe')
print(results_df.sort_values('sharpe', ascending=False)[:10])
 
print('-' * 50)
print('Sort by annualized_returns')
print(results_df.sort_values('annualized_returns', ascending=False)[:10])

from rqdatac import *

In [20]:
data = []
for result, name in results:
    summary = result['summary']
    data.append({
        'name': name,
        'annualized_returns': summary['annualized_returns'],
        'sharpe': summary['sharpe'],
        'max_drawdown': summary['max_drawdown'],
    })  

In [25]:
# 分析回测
results_df = pd.DataFrame(data)
 
print('')
print('-' * 50)
print('Sort by sharpe')
print(results_df.sort_values('sharpe', ascending=False)[:10])
 
print('-' * 50)
print('Sort by annualized_returns')
print(results_df.sort_values('annualized_returns', ascending=False)[:10])

from rqdatac import *


--------------------------------------------------
Sort by sharpe
    annualized_returns  max_drawdown                      name  sharpe
38               0.519         0.165  0.3-0.8000000000000003-6   1.676
60               0.299         0.124                 0.4-0.4-7   1.675
63               0.468         0.226                 0.4-0.5-3   1.656
11               0.383         0.215                 0.3-0.4-7   1.651
14               0.476         0.248                 0.3-0.5-3   1.628
10               0.405         0.226                 0.3-0.4-6   1.608
21               0.470         0.236  0.3-0.6000000000000001-3   1.601
15               0.443         0.243                 0.3-0.5-4   1.588
59               0.327         0.187                 0.4-0.4-6   1.584
31               0.453         0.156  0.3-0.7000000000000002-6   1.581
--------------------------------------------------
Sort by annualized_returns
    annualized_returns  max_drawdown                      name  sharpe
38 